In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import re

# 저장 이름 설정
OUTPUT_FILE = "chosun_articles_sobikupon.json"

# 검색 결과 URL 생성
# 1번째 페이지와 그 이후 페이지 url 구성이 달라 조건문으로 분리
def get_search_url(page):
    if page == 1:
        return "https://www.chosun.com/nsearch/?query=소비쿠폰"
    else:
        return (
            f"https://www.chosun.com/nsearch/?query=소비쿠폰"
            f"&page={page}"
            f"&siteid=&sort=1&date_period=all&date_start=&date_end="
            f"&writer=&field=&emd_word=&expt_word=&opt_chk=false&app_check=0"
            f"&website=www,chosun&category="
        )

# 기사 본문 추출 함수
def get_article_content(article_url, soup):
    content = ""
    ## url도메인마다 태그가 달라 이를 분리해줌
    ## 총 3가지로 분리
    if article_url.startswith("https://woman.chosun.com/"):
        article_tag = soup.select_one("article")
        if article_tag:
            p_tags = article_tag.find_all("p")
            content = "\n".join(p.get_text(strip=True) for p in p_tags if p.get_text(strip=True))

    elif article_url.startswith("https://weekly.chosun.com/") or \
         article_url.startswith("https://topclass.chosun.com/") or \
         article_url.startswith("https://it.chosun.com/"):
        article_tag = soup.select_one("article#article-view-content-div")
        if article_tag:
            p_tags = article_tag.find_all("p")
            content = "\n".join(p.get_text(strip=True) for p in p_tags if p.get_text(strip=True))

    else:
        content_tags = soup.select("p.article-body__content.article-body__content-text")
        content = "\n".join(p.get_text(strip=True) for p in content_tags if p.get_text(strip=True))

    return content

# 날짜 수집 함수
def get_article_date(article_url, soup):
    date_text = ""
    try:
        if article_url.startswith("https://woman.chosun.com/"):
            li = soup.select_one("div.info-group article.item ul.infomation li:nth-of-type(2)")
            if li:
                date_text = li.get_text(strip=True)

        elif article_url.startswith("https://weekly.chosun.com/") or \
             article_url.startswith("https://topclass.chosun.com/"):
            li = soup.select_one("div.head-inner ul.infomation li:nth-of-type(2)")
            if li:
                date_text = li.get_text(strip=True)

        elif article_url.startswith("https://it.chosun.com/"):
            li = soup.select_one("ul.infomation li:nth-of-type(2)")
            if li:
                date_text = li.get_text(strip=True)

        else:
            date_span = soup.select_one("span.inputDate")
            if date_span:
                date_text = date_span.get_text(strip=True)
    except Exception as e: #예외처리
        print(f"날짜 수집 중 오류: {e}")

    return date_text

# 브라우저 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.set_page_load_timeout(180)  

all_articles = []
visited_urls = set()

# 1~20페이지 반복
for page in range(21, 22):
    url = get_search_url(page)
    print(f"\n[페이지 {page}] {url}")
    try:
        driver.get(url)
    except Exception as e:
        print(f" 페이지 {page} 로딩 실패: {e}")
        continue

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.story-card__headline > a"))
        )
    except:
        print(f" 페이지 {page}: 기사 로딩 실패")
        continue

    soup = BeautifulSoup(driver.page_source, "html.parser")
    link_tags = soup.select("div.story-card__headline > a")

    if not link_tags:
        print(f" 페이지 {page}에서 기사 없음")
        continue

    for tag in link_tags:
        try:
            title = tag.get_text(strip=True)
            article_url = tag["href"]
            if not article_url.startswith("http"):
                article_url = "https://www.chosun.com" + article_url

            #  archive URL 스킵 조건 추가
            if article_url.startswith("https://archive.chosun.com"):
                print(f"⏩ archive URL 스킵됨: {article_url}")
                continue

            if article_url in visited_urls:
                continue
            visited_urls.add(article_url)

            # 기사 본문 페이지 이동 및 수집
            try:
                driver.get(article_url)
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "article, p.article-body__content"))
                )
            except Exception as e:
                print(f"  기사 페이지 로딩 실패 (타임아웃 또는 로딩 실패): {e} | URL: {article_url}")
                continue

            time.sleep(2.5)
            article_soup = BeautifulSoup(driver.page_source, "html.parser")

            # 본문 및 날짜 내용 추출
            content = get_article_content(article_url, article_soup)
            date = get_article_date(article_url, article_soup)

            # 저장
            article_data = {
                "title": title,
                "url": article_url,
                "date": date,
                "content": content
            }
            all_articles.append(article_data)

            # 출력
            content_preview = content[:10].replace("\n", " ").strip()
            print(f" 수집됨 | URL: {article_url}")
            print(f"제목: {title}")
            print(f"날짜: {date}")
            print(f"본문: {content_preview}...\n")

        except Exception as e:
            print(f" 오류 발생: {e}")

# 브라우저 종료 및 저장
driver.quit()

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=2)

print(f"\n 총 {len(all_articles)}개의 기사 저장 완료: {OUTPUT_FILE}")



[페이지 21] https://www.chosun.com/nsearch/?query=소비쿠폰&page=21&siteid=&sort=1&date_period=all&date_start=&date_end=&writer=&field=&emd_word=&expt_word=&opt_chk=false&app_check=0&website=www,chosun&category=
✅ 수집됨 | URL: https://www.chosun.com/politics/politics_general/2025/06/20/66UVA2RPVZE3TEPIQNBXXMYIDU/
제목: 재래시장 깜짝 방문한 李대통령 “주가 많이 올랐죠?”
날짜: 입력 2025.06.20. 18:10
본문: 이재명 대통령이 2...

✅ 수집됨 | URL: http://woman.chosun.com/news/articleView.html?idxno=121608
제목: [키워드로 읽는 경제 트렌드] 새 정부소비쿠폰, 누가 언제 얼마 받나
날짜: 
본문: ...

✅ 수집됨 | URL: https://biz.chosun.com/stock/stock_general/2025/06/20/DZX5KBSWL5DWDEREQX3SKLSSAY/
제목: [마켓뷰] 30兆 추경, 내수 살릴까… 중동 리스크에도 3000선 뚫은 코스피
날짜: 입력 2025.06.20. 16:33
본문: 코스피지수가 202...

✅ 수집됨 | URL: https://biz.chosun.com/policy/policy_sub/2025/06/20/UXYY6H2AKNH55ALGNXRRKUTAVI/
제목: [새정부 추경]소비쿠폰에 10兆, AI 전환에 투자… 새 정부 추경, 필수 추경과 차이점은
날짜: 입력 2025.06.20. 16:24
본문: 이재명 정부의 30...

✅ 수집됨 | URL: https://biz.chosun.com/policy/politics/president_office/2025/06/20/KPKOYFIDMZEMNFFRRPZGQOQXWU/
